In [1]:
import pandas as pd

In [89]:
class Table(object):
    
    def __init__(self,ds_name,table_name,columns=None,index_col=None):
        self.ds_name = ds_name
        self.table_name = table_name
        self.file_name = None
        self.columns = columns
        self.index_col = index_col
        self.data = pd.DataFrame()
        
    def _file_name(self):
        return self.ds_name + '.d/' + self.table_name + '.csv'
        
    def read(self,file_name=None):
        if not file_name:
            file_name = self._file_name()
        self.file_name = file_name
        try:
            self.data = pd.read_csv(file_name,usecols=self.columns,index_col=self.index_col)
        except ValueError as err:
            msg = err.args[0]
            if msg.endswith('is not in list'):
                c = msg.split("'")[1]
                raise ValueError("'{}' is not in the set of columns in file '{}'".format(c,file_name))
            if msg.startswith('Index') and msg.endswith('invalid'):
                raise ValueError("Index column '{}' is not in the set of columns in file '{}'".format(self.index_col,file_name))
            raise
        return self.data
    
    def write(self,file_name=None,precision=None,index=False):
        if not file_name:
            file_name = self.file_name
        if not file_name:
            file_name = self._file_name()
        self.file_name = file_name
        float_format = None
        if precision is not None:
            float_format = '%.{:d}g'.format(precision)
        self.data.to_csv(file_name,index=index,float_format=float_format)

In [90]:
t = Table('frame-6','nodes',columns=['NODEID','X','Y'])

In [91]:
t.read()

,NODEID,X,Y
0,A,0,0
1,B,0,4000
2,C,8000,4000
3,D,8000,0


In [86]:
t.data.loc[:,'X'] /= 3.
t.data

,NODEID,X,Y
0,A,0.000000,0
1,B,0.000000,4000
2,C,2666.666667,4000
3,D,2666.666667,0


In [88]:
t.write('xxx.csv',precision=6)